In [1]:
#!/usr/bin/env python3  
# -*- coding: utf-8 -*-  
"""  
 @Author: HUJING
 @Time:6/13/18 9:15 AM 2018
 @Email: jhsa26@mail.ustc.edu.cn
 @Site:jhsa26.github.io
 """
import numpy as np
import matplotlib.pyplot as plt
import os

def get_filename(filepath_disp_training,filepath_vs_training,dataset_type):
    filename_dispersion_total = []
    filename_vs_total = []
    key_total = []
    if dataset_type == "train":
        if os.path.exists(filepath_disp_training) and os.path.exists(filepath_vs_training):

            files_disp = os.listdir(filepath_disp_training)

            # read inputs
            for file in files_disp:

                key = file[2:-4]  # add group disp and phase disp
#                 print(key)
                filename_disp = filepath_disp_training + file

                filename_vs = filepath_vs_training + file
#                 print(filename_vs)
                if os.path.exists(filename_vs) and os.path.exists(filename_disp):
                    filename_dispersion_total.append(filename_disp)

                    filename_vs_total.append(filename_vs)

                    key_total.append(file[0:-4])

            return np.array(filename_dispersion_total), np.array(filename_vs_total), np.array(key_total)
        else:
            print('Input file path is not exist, check the input path!')
            return None, None, None


def gaussian_map(vel,vel_axis,radius=0.1):
    rows  = vel_axis.shape[0]
    cols  = vel.shape[0]
    vel_map = np.zeros((rows,cols))
    for i in range(cols):
        vel_temp = vel[i]
         
        if  isinstance(radius,(list, tuple, np.ndarray)):
            x_gaussian = gaussian(vel_temp, vel_axis, r=radius[i])
            vel_map[:, i] = x_gaussian
        else: 
            x_gaussian = gaussian(vel_temp, vel_axis, r=radius)
            vel_map[:, i] = x_gaussian
            
    return vel_map

def gaussian(vel,vel_axis,r=0.1):
    x_gaussian = np.exp(-((vel_axis-vel)**2)/(2*r**2))
    return x_gaussian



filepath_disp_training = './disp_out/'
filepath_vs_training =  './vs_out/'
os.system("rm -rf disp_combine_gaussian_map  && mkdir disp_combine_gaussian_map  ")
use_uncertainty=3
if use_uncertainty ==1: # for all periods we take an average or median uncertianty
    r_p = 0.04111930355425 
    r_g = 0.07261627539995
elif use_uncertainty==2: # for each period,we take an average or median uncertainty
    r_p =[0.09042569,   0.079493 ,  0.07365631,   0.06447944, 0.06076378, 0.05558007,
                        0.0447768,    0.040191,   0.03521833, 0.03173376, 0.02592121,
          0.02728111,  0.02891647,  0.03051841,   0.03277794, 0.03375802, 0.03446512]

    r_g=[0.11770464, 0.11551448, 0.10761005, 0.10085003, 0.09631512, 0.09072495,
                     0.07843351, 0.07537709, 0.06930856, 0.06507   , 0.0517338,
         0.04895614, 0.0481836 ,  0.0479863, 0.05141948, 0.05516992, 0.05765841]
elif use_uncertainty==3:
    r_p = 0.1
    r_g = 0.1
disp_names,vs_names,keys_names = get_filename(filepath_disp_training,filepath_vs_training,'train')

vel_axis = np.linspace(1,4.5,num=60)  # velocity range, vmin=1;vmax=4.5, vmin should smaller than the minimum Vs value of training set.
fig=plt.figure()

In [2]:
count = 0
delete_point=[]
save_point = []
for i in range(len(keys_names)):
    try:
        
        vel_pg=np.loadtxt(disp_names[i])
    except:
        print(disp_names[i]," error")
        print(i,len(keys_names))
    else:
        freqs  = vel_pg[:,0] #vel_pg = vel_pg[:,1:3]     # np.delete(a,0,axis=1)
        vel_p = vel_pg[:,1]
        vel_g = vel_pg[:,2]
        vs_syn=np.loadtxt(vs_names[i])
        depths = vs_syn[:,0]
        vel_vs = vs_syn[:,1]
        if min(vs_syn[:,1])>1 and min(vel_g[:])>1. and min(vel_p[:])>1:
            count = count + 1
            vel_map_p= gaussian_map(vel_p,vel_axis,radius=r_p)
            vel_map_g= gaussian_map(vel_g,vel_axis,radius=r_g)
            vs_map_syn = gaussian_map(vel_vs,vel_axis,radius=0.1)
        #     name_p = './disp_gaussian_map/p_' + str(lat)+"_"+str(lon) + '.npy'
        #     name_g = './disp_gaussian_map/g_' + str(lat)+"_"+str(lon) + '.npy'
            name_pg = './disp_combine_gaussian_map/'+ keys_names[i] + '.npy'
#             name_vel ='./vs_syn_gaussian/' + keys_names[i] + '.npy' 
            np.save(name_pg,np.array([vel_map_p,vel_map_g]))
        #     np.save(name_p,vel_map_p)
        #     np.save(name_g,vel_map_g)
#             np.save(name_vel,vs_map_syn)
#             plt.subplot(1,3,1)
#             plt.imshow(vel_map_p,aspect='auto',origin='lower',extent=[2.5,16,1,5])
            
#             plt.subplot(1,3,2)
#             plt.title(disp_names[i])
#             plt.imshow(vel_map_g,aspect='auto',origin='lower',extent=[2.5,16,1,5])
#             plt.subplot(1,3,3)
#             plt.imshow(vs_map_syn.T,aspect='auto',origin='upper',extent=[1,5,47,0])
#             plt.tight_layout()
#             plt.pause(0.01)
#             fig.clear()
            iter,lat,lon=keys_names[i].split("_")
#             save_point.append([float(lat),float(lon)])
#             print(count)
        else:
            iter,lat,lon=keys_names[i].split("_")
#             print(count,latlon)
            delete_point.append([float(lat),float(lon)])
            
print(count)   

22580


In [3]:
figure = plt.figure(figsize=(10,5))
delete_point= np.array(delete_point)
save_point= np.array(save_point)
plt.subplot(1,2,1)
plt.plot(delete_point[:,1],delete_point[:,0],".")
plt.title("delete")
plt.subplot(1,2,2)
plt.plot(save_point[:,1],save_point[:,0],".")
plt.title("save")
plt.pause(0.1)

IndexError: too many indices for array